# Day 17 - o1-preview (GHC)

In [1]:
def main():
    # Read the input from 'input.txt'
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()
    
    # Initialize registers and program
    registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            # Parse register values
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            registers[reg_name] = value
        elif line.startswith('Program:'):
            # Parse program
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]
    
    # Initialize instruction pointer and output list
    ip = 0  # instruction pointer
    outputs = []

    # Start execution
    while ip < len(program):
        opcode = program[ip]
        if ip + 1 < len(program):
            operand = program[ip + 1]
        else:
            # If there's no operand, halt
            break
        
        # Instruction pointer increment (assumed to be +2 by default)
        ip_increment = 2

        if opcode == 0:  # adv
            # Operand type: combo operand
            value = get_combo_operand_value(operand, registers)
            denominator = pow(2, value)
            numerator = registers['A']
            result = numerator // denominator
            registers['A'] = result
        elif opcode == 1:  # bxl
            # Operand type: literal operand
            value = operand
            registers['B'] = registers['B'] ^ value
        elif opcode == 2:  # bst
            # Operand type: combo operand
            value = get_combo_operand_value(operand, registers)
            result = value % 8
            registers['B'] = result
        elif opcode == 3:  # jnz
            # Operand type: literal operand
            value = operand
            if registers['A'] != 0:
                ip = value
                ip_increment = 0  # Don't add 2 to ip
        elif opcode == 4:  # bxc
            # Operand type: ignored
            registers['B'] = registers['B'] ^ registers['C']
        elif opcode == 5:  # out
            # Operand type: combo operand
            value = get_combo_operand_value(operand, registers)
            output_value = value % 8
            outputs.append(str(output_value))
        elif opcode == 6:  # bdv
            # Operand type: combo operand
            value = get_combo_operand_value(operand, registers)
            denominator = pow(2, value)
            numerator = registers['A']
            result = numerator // denominator
            registers['B'] = result
        elif opcode == 7:  # cdv
            # Operand type: combo operand
            value = get_combo_operand_value(operand, registers)
            denominator = pow(2, value)
            numerator = registers['A']
            result = numerator // denominator
            registers['C'] = result
        else:
            # Invalid opcode, halt
            break

        # Increment instruction pointer
        if ip_increment:
            ip += ip_increment
        else:
            # ip was modified by the instruction (e.g., jnz)
            pass

    # Output the collected outputs
    print(','.join(outputs))


def get_combo_operand_value(operand, registers):
    if operand == 7:
        raise ValueError("Invalid combo operand 7")
    elif 0 <= operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
        raise ValueError(f"Invalid operand value: {operand}")


if __name__ == '__main__':
    main()

3,4,3,1,7,6,5,6,0


## Part 2

In [2]:
def main():
    # Read the input from 'input.txt'
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()

    # Initialize registers and program
    initial_registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            # Parse register values
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            if reg_name != 'A':
                initial_registers[reg_name] = value
            # We'll find the correct A
        elif line.startswith('Program:'):
            # Parse program
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]

    # Start searching for the lowest positive A
    A = 1
    max_A = 1 << 24  # Set an upper limit to prevent infinite loop
    while A < max_A:
        registers = initial_registers.copy()
        registers['A'] = A
        outputs = run_program(program, registers)
        if outputs == program:
            print(A)
            return
        A += 1

    print("No valid A found within the limit.")

def run_program(program, registers):
    outputs = []
    ip = 0  # Instruction pointer
    while ip < len(program):
        opcode = program[ip]
        if ip + 1 >= len(program):
            break  # If there's no operand, halt
        operand = program[ip + 1]
        ip_increment = 2  # Default IP increment

        if opcode == 0:  # adv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['A'] = registers['A'] // denominator
        elif opcode == 1:  # bxl
            value = operand  # Literal operand
            registers['B'] ^= value
        elif opcode == 2:  # bst
            value = get_combo_operand_value(operand, registers)
            registers['B'] = value % 8
        elif opcode == 3:  # jnz
            value = operand  # Literal operand
            if registers['A'] != 0:
                ip = value
                ip_increment = 0
        elif opcode == 4:  # bxc
            # Reads operand but ignores it
            registers['B'] ^= registers['C']
        elif opcode == 5:  # out
            value = get_combo_operand_value(operand, registers)
            outputs.append(value % 8)
        elif opcode == 6:  # bdv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['B'] = registers['A'] // denominator
        elif opcode == 7:  # cdv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['C'] = registers['A'] // denominator
        else:
            # Invalid opcode, halt
            break

        if ip_increment:
            ip += ip_increment
    return outputs

def get_combo_operand_value(operand, registers):
    if operand == 7:
        raise ValueError("Invalid combo operand 7")
    elif operand == 0:
        return 0
    elif operand == 1:
        return 1
    elif operand == 2:
        return 2
    elif operand == 3:
        return 3
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
        raise ValueError(f"Invalid operand: {operand}")

if __name__ == '__main__':
    main()

KeyboardInterrupt: 

In [3]:
def main():
    # Read the input from 'input.txt'
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()
    
    # Initialize registers and program
    initial_registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            # Parse register values
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            if reg_name != 'A':
                initial_registers[reg_name] = value
            # We'll calculate A later
        elif line.startswith('Program:'):
            # Parse program
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]
    
    # The outputs should match the program
    required_outputs = program.copy()
    
    # Reconstruct A from the outputs
    A = reconstruct_A_from_outputs(required_outputs)
    print(A)

def reconstruct_A_from_outputs(outputs):
    # Since the program outputs A % 8 after dividing A by 8,
    # we can reconstruct A by reversing the outputs
    outputs_reversed = outputs[::-1]
    A = 0
    for value in outputs_reversed:
        A = A * 8 + value
    return A

if __name__ == '__main__':
    main()

16339543718498


In [4]:
def main():
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()
    
    # Initialize registers and program
    initial_registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            if reg_name != 'A':
                initial_registers[reg_name] = value
        elif line.startswith('Program:'):
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]
    
    # Reconstruct A from the outputs
    A_over_8 = reconstruct_A_from_outputs(program)
    
    # Minimal positive initial A
    initial_A = A_over_8 * 8 + 1
    print(initial_A)

def reconstruct_A_from_outputs(outputs):
    outputs_reversed = outputs[::-1]
    A = 0
    for value in outputs_reversed:
        A = A * 8 + value
    return A

if __name__ == '__main__':
    main()

130716349747985


In [5]:
def main():
    # Read the input from 'input.txt'
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()

    # Initialize registers and program
    initial_registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            # Parse register values
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            if reg_name != 'A':
                initial_registers[reg_name] = value
            # We'll find the correct A
        elif line.startswith('Program:'):
            # Parse program
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]

    # Extract the outputs expected (the program itself)
    expected_outputs = program.copy()

    # Start searching for the lowest positive A
    max_A = 1 << 24  # Upper limit to prevent long runtime
    for A in range(1, max_A):
        registers = initial_registers.copy()
        registers['A'] = A
        outputs = run_program(program, registers)
        if outputs == expected_outputs:
            print(A)
            return

    print("No valid A found within the limit.")

def run_program(program, registers):
    outputs = []
    ip = 0  # Instruction pointer
    while ip < len(program):
        opcode = program[ip]
        if ip + 1 >= len(program):
            break  # If there's no operand, halt
        operand = program[ip + 1]
        ip_increment = 2  # Default IP increment

        if opcode == 0:  # adv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['A'] = registers['A'] // denominator
        elif opcode == 1:  # bxl
            value = operand  # Literal operand
            registers['B'] ^= value
        elif opcode == 2:  # bst
            value = get_combo_operand_value(operand, registers)
            registers['B'] = value % 8
        elif opcode == 3:  # jnz
            value = operand  # Literal operand
            if registers['A'] != 0:
                ip = value
                ip_increment = 0
        elif opcode == 4:  # bxc
            # Reads operand but ignores it
            registers['B'] ^= registers['C']
        elif opcode == 5:  # out
            value = get_combo_operand_value(operand, registers)
            outputs.append(value % 8)
        elif opcode == 6:  # bdv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['B'] = registers['A'] // denominator
        elif opcode == 7:  # cdv
            value = get_combo_operand_value(operand, registers)
            denominator = 2 ** value
            registers['C'] = registers['A'] // denominator
        else:
            # Invalid opcode, halt
            break

        if ip_increment:
            ip += ip_increment
    return outputs

def get_combo_operand_value(operand, registers):
    if operand == 7:
        raise ValueError("Invalid combo operand 7")
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    elif 0 <= operand <= 3:
        return operand
    else:
        raise ValueError(f"Invalid operand: {operand}")

if __name__ == '__main__':
    main()

No valid A found within the limit.


In [6]:
def main():
    # Read the input from 'input.txt'
    with open('input.txt', 'r') as f:
        lines = f.read().splitlines()

    # Initialize registers and program
    initial_registers = {'A': 0, 'B': 0, 'C': 0}
    program = []
    for line in lines:
        line = line.strip()
        if line.startswith('Register'):
            # Parse register values
            reg_name, value = line.split(':')
            reg_name = reg_name.strip().split()[1]
            value = int(value.strip())
            if reg_name != 'A':
                initial_registers[reg_name] = value
            # We'll calculate A later
        elif line.startswith('Program:'):
            # Parse program
            _, prog = line.split(':', 1)
            program = [int(x.strip()) for x in prog.strip().split(',')]

    # Reconstruct A from the outputs
    A_over_8 = reconstruct_A_from_outputs(program)
    initial_A = A_over_8 * 8
    print(initial_A)

def reconstruct_A_from_outputs(outputs):
    outputs_reversed = outputs[::-1]
    A = 0
    for value in outputs_reversed:
        A = A * 8 + value
    return A

if __name__ == '__main__':
    main()

130716349747984
